In [62]:
import requests, bs4
import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pprint
import re

In [63]:
evo_links = ["https://www.evo.com/shop/ski/skis/rpp_400", "https://www.evo.com/shop/ski/skis/p_2/rpp_400", "https://www.evo.com/shop/ski/skis/p_3/rpp_400"]

In [64]:
ski_list= []
for link in evo_links:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0', 'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                                           'accept-encoding': 'gzip, deflate, br',
                                           'accept-language': 'en-US,en;q=0.9',
                                           'cache-control': 'max-age=0',
                                           'cookie': 'audS=t; RF_BROWSER_ID=43AAS-RITTCuof2YtJymvA; _gcl_au=1.1.722715582.1629050143; G_ENABLED_IDPS=google; _fbp=fb.1.1629050143930.1251742572; run_fs_for_user=32; RF_VISITED=true; g_state={"i_p":1633118087072,"i_l":3}; sortOrder=1; fbm_161006757273279=base_domain=.redfin.com; RF_LAST_USER_ACTION=1632514720839%3A3cea684cac5f0a20e2296fbf3d10735dd534ba65; RF_PARTY_ID=13374233; RF_AUTH=e272327ca9e9514c5f9cbfdd96f30d90ca753ec2; RF_W_AUTH=e272327ca9e9514c5f9cbfdd96f30d90ca753ec2; RF_SECURE_AUTH=0a7a875e056bea00e8fed40db13b26b4a4903ce3; RF_ACCESS_LEVEL=3; JSESSIONID=9F42CA463F442F28069D7E4DF3520BC2; RF_LAST_ACCESS=1632514768392%3Acb9b7b64e6f57e2b9bd8b52d511728ecedc1c24f; RF_BUSINESS_MARKET=12; RF_MARKET=chicago; _gcl_aw=GCL.1634505347.CjwKCAjwk6-LBhBZEiwAOUUDp7vilByLZ1CTaj2MDO1VOF77qRdJMzWsNtFHD8LYcdp45hEdYnDcdBoCyXoQAvD_BwE; _gcl_dc=GCL.1634505347.CjwKCAjwk6-LBhBZEiwAOUUDp7vilByLZ1CTaj2MDO1VOF77qRdJMzWsNtFHD8LYcdp45hEdYnDcdBoCyXoQAvD_BwE; _gac_UA-294985-1=1.1634505348.CjwKCAjwk6-LBhBZEiwAOUUDp7vilByLZ1CTaj2MDO1VOF77qRdJMzWsNtFHD8LYcdp45hEdYnDcdBoCyXoQAvD_BwE; RF_LAST_NAV=0; sortOption=list_price; RF_BID_UPDATED=1; _fbc=fb.1.1635176555257.IwAR3jbeg-Yr0A2_SLjzolmvaQIu9W539x9giU95K9ApBUYUEHum8dHVSbHZE; ki_r=; RF_LDP_VIEWS_FOR_PROMPT=%7B%22viewsData%22%3A%7B%2210-09-2021%22%3A%7B%7D%2C%2210-13-2021%22%3A%7B%7D%2C%2210-16-2021%22%3A%7B%7D%2C%2210-17-2021%22%3A%7B%7D%2C%2210-18-2021%22%3A%7B%7D%2C%2210-19-2021%22%3A%7B%7D%2C%2210-25-2021%22%3A%7B%7D%2C%2210-28-2021%22%3A%7B%7D%7D%2C%22expiration%22%3A%222023-08-15T17%3A56%3A27.641Z%22%2C%22totalPromptedLdps%22%3A0%7D; unifiedLastSearch=name%3DWilmette%26subName%3DWilmette%252C%2520IL%252C%2520USA%26url%3D%252Fcity%252F20429%252FIL%252FWilmette%26id%3D2_20429%26type%3D2%26unifiedSearchType%3D2%26isSavedSearch%3D%26countryCode%3DUS; ki_t=1629050185221%3B1635458424530%3B1635465309003%3B12%3B120; RF_HOT_MARKET=%7B%22lastPromptedDate%22%3A1635465310710%2C%22expiration%22%3A1636070110710%2C%22promptedTimes%22%3A1%7D; RF_LISTING_VIEWS=137875645.140560512.140411917.139597233.139732219.139761763.135994116.135877651.133846633.101296437.128478937.139204392.139277511.139660824.136187462.138916803.139675098.139680422.139679117.139683635; RF_LAST_DP_SERVICE_REGION=2039; RF_LAST_SEARCHED_CITY=Wilmette; RF_UNBLOCK_ID=aAXo4vWj; AKA_A2=A; RF_CORVAIR_LAST_VERSION=390.4.0; bm_mi=A5E4D781245CB153CFC0C5A6E687F628~LUqnyd2IOoWYApetm585Li4uHU8l3PeUqpI/QJFB3qyVV7k+aWkxUmUNONL3SEzTqJyDmn+0ANehA7KHP0cnRgL9tnmN6OEIY6sm9QHN8QgzGwfJP39Sl2p6Kmvs5FODZ3+OANYgTI0wqgi9Ad5d+5cdZZDZQuGP0MRqgN1u5O9d2aDMKtdud88reLP2YlnuLy1qr3xVZ3mIjpp3+DTBp9z1NM0px8GjfNnGXxKzDXbNlsIYLpS8MHD993txUAJp8DMB9giwNjiSUvlomU5Z81hIY/GCUUrDYwtW7mlTgPzsfkY2hixo8hASnVRJdJKh4AJkAekI9zsTmBwNGUQJyVkMoRinqip0pl+ON48z/CngNByUNiklK9XBz22pHCHeIJD4GkM9FwF9cdCCFEPpFQ==; ak_bmsc=AC6F3B3FA9F349985AD2D4C3359A812C~000000000000000000000000000000~YAAQJcLFFxUbpdl8AQAA69y43Q3cm/YlXAY9N+lUB/ymC2MFpU9ZFU83GeQMg04fAVf2iR5jZxfEoS6oQ7CqbKfF/JNGDjjYwFGFfbMAMHecxgR6JUL0tWKevcknhPuonDDpbnlvpk+WFRuCODlNz3f6wEBZsfOo6thw8mVLPCfw7ADH+AhvT8cFQeG3wbGS8Y/4qAUH4Sdl+IRf0fposOiwByssBYHW2hjZwLArL4IN194HHFPDc24fHZV6P5hL86Eb44rq0/CJNMKxZTd1rNhJYDZYmyDSSV6JT0bJn2AJXZxLTHTyCazNQ2cn4TsUN+phF9JAPpUwvZaZ9SzU6lo3KQ5Ng2BnWNj56DVH/HiA9qJCxYRHUAq95nvVN4gDVUzKELzjW6vqBEPO9GjdIUT+1KXBFf0=; _ga_928P0PZ00X=GS1.1.1635806984.31.1.1635807451.60; _uetsid=18aa30303b6711ec942d53fafedf1211; _uetvid=02e6f3e0fdf211ebb2f3399bb0a000f8; AMP_TOKEN=%24NOT_FOUND; _ga=GA1.2.1450004020.1629050144; _gid=GA1.2.526642902.1635807452; userPreferences=parcels%3Dtrue%26schools%3Dfalse%26mapStyle%3Ds%26statistics%3Dtrue%26agcTooltip%3Dfalse%26agentReset%3Dfalse%26ldpRegister%3Dfalse%26afCard%3D2%26schoolType%3D0%26lastSeenLdp%3DnoSharedSearchCookie%26viewedSwipeableHomeCardsDate%3D1635807452561; FEED_COUNT=71%3At; RF_BROWSER_CAPABILITIES=%7B%22screen-size%22%3A2%2C%22events-touch%22%3Afalse%2C%22ios-app-store%22%3Afalse%2C%22google-play-store%22%3Afalse%2C%22ios-web-view%22%3Afalse%2C%22android-web-view%22%3Afalse%7D; bm_sv=1325C498ACBD29CE779E41FE66ED22EC~yuqVjd43aTJXIlu+m9YNpkfDLMtD9VHeCM2DuE6FuTTgLiux+AVds8ZsfBLBTsPp9+AFDb6Xf7bMZbAakNn7rzTGrPur7nJ3jlaA/Lzqa36zNyh8u7/EmrSwDWt6Qx/N/C8mfwBwDz+i/m6FWVYtE4EDZRSEUiLeOt/6Kghx0go=',
                                           'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
                                           'sec-ch-ua-mobile': '?0',
                                           'sec-ch-ua-platform': '"macOS"',
                                           'sec-fetch-dest': 'document',
                                           'sec-fetch-mode': 'navigate',
                                           'sec-fetch-site': 'same-origin',
                                           'sec-fetch-user': '?1',
                                           'upgrade-insecure-requests': '1', 
                                           'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'})
    soup= bs(response.content, "html.parser")
    for div in soup.find_all(class_= 'product-thumb-details'):
            ski_name = div.find_all(class_='product-thumb-title')
            #venue_location = div.find_all(class_='venueCard--locationText')
            ski_price = div.find_all(class_= "product-thumb-price")
            detail_link = div.find('a', href = True) 
            ski_list.append({"name": ski_name[0].text.strip(), "price": ski_price[0].text.replace("$", "").replace(",", "").strip(),'detail_link': detail_link['href'].strip()})
    ski_list

In [65]:
ski_list_df= pd.DataFrame(ski_list)
ski_list_df

,name,price,detail_link
0,Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr
1,Rossignol Black Ops Blazer Skis - Women's 2022,699.95,/skis/rossignol-black-ops-blazer-womens
2,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens
3,Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...
4,Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...
...,...,...,...
874,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100
875,Atomic Bent Chetler Jr Skis - Little Kids' 202...,199.99,/used/skis/atomic-bent-chetler-jr-little-kids
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens
877,Fischer Ranger 99 Ti Skis ​+ Tyrolia Attack2 1...,1048.95,/outlet/used/ski-packages/fischer-ranger-99-ti...


In [66]:
brands = ["Armada", "Atomic", "Stöckli", "Black Crows", "Black Diamond", "Blizzard", "Coalition Snow", "DPS", "Dynafit", "Dynastar", "Elan", "Faction", "Fischer", "Folsom Skiis", "Folsom", "G3", "Head", "Icelantic", "K2", "Lib Tech", "Liberty", "Line Skis", "Movement", "Nordica", "ON3P", "RMU", "Rossignol", "Salomon", "Scott", "Season", "WNDR Alpine", "Völkl", "ZAG"]

In [67]:
def find_brand_list(text, brand_list):
    for brand in brand_list:
        if brand in text:
            return brand 
        else:
            continue
    return 0
    

In [68]:
ski_list_df["brand"]= ski_list_df["name"].apply(lambda x: find_brand_list(x, brands))
ski_list_df

,name,price,detail_link,brand
0,Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer
1,Rossignol Black Ops Blazer Skis - Women's 2022,699.95,/skis/rossignol-black-ops-blazer-womens,Rossignol
2,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol
3,Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head
4,Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head
...,...,...,...,...
874,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic
875,Atomic Bent Chetler Jr Skis - Little Kids' 202...,199.99,/used/skis/atomic-bent-chetler-jr-little-kids,Atomic
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon
877,Fischer Ranger 99 Ti Skis ​+ Tyrolia Attack2 1...,1048.95,/outlet/used/ski-packages/fischer-ranger-99-ti...,Fischer


In [69]:
years = ["2022", "2021", "2020", "2019", "2018"]

In [70]:
def find_year_list(text, year_list):
    for year in year_list:
        if year in text:
            return year 
        else:
            continue
    return 0
    

In [71]:
ski_list_df["year"]= ski_list_df["name"].apply(lambda x: find_year_list(x, years))
ski_list_df

,name,price,detail_link,brand,year
0,Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022
1,Rossignol Black Ops Blazer Skis - Women's 2022,699.95,/skis/rossignol-black-ops-blazer-womens,Rossignol,2022
2,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022
3,Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022
4,Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022
...,...,...,...,...,...
874,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022
875,Atomic Bent Chetler Jr Skis - Little Kids' 202...,199.99,/used/skis/atomic-bent-chetler-jr-little-kids,Atomic,2022
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0
877,Fischer Ranger 99 Ti Skis ​+ Tyrolia Attack2 1...,1048.95,/outlet/used/ski-packages/fischer-ranger-99-ti...,Fischer,0


In [72]:
ski_list_df[ski_list_df["year"]== 0]

,name,price,detail_link,brand,year
13,Atomic Vantage 79 Ti Skis ​+ F 12 GW Bindings,599.00,/outlet/ski-packages/atomic-vantage-79-ti-skis...,Atomic,0
14,Völkl M5 Mantra Skis,699.99,/outlet/skis/volkl-m5-mantra,Völkl,0
16,Salomon QST Lux 92 Skis - Women's,549.99,/outlet/skis/salomon-qst-lux-92-womens-2020,Salomon,0
24,Nordica Enforcer 88 Skis,649.99,/outlet/skis/nordica-enforcer-88-2021,Nordica,0
25,Faction Candide 2.0 Skis,699.00,/outlet/skis/faction-candide-20,Faction,0
...,...,...,...,...,...
871,Folsom Skis Cash 106 Skis - Women's,799.95,/outlet/skis/folsom-cash-106-womens,Folsom,0
872,Atomic Punx Jr Skis - Little Boys',199.99,/outlet/skis/atomic-punx-jr-little-boys-2021,Atomic,0
873,Armada ARW 116 VJJ UL Skis ​+ Marker Duke PT 1...,1349.94,/outlet/used/ski-packages/armada-arw-116-vjj-u...,Armada,0
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0


In [73]:
genders = ["Women's", "Mens", "Boys", "Girls", "Toddlers", "Junior", "Jr", "Kids", "Unisex"]

In [74]:
def find_gender_list(text, gender_list):
    for gender in gender_list:
        if gender in text:
            return gender 
        else:
            continue
    return "Unisex"

In [75]:
ski_list_df["gender"]= ski_list_df["name"].apply(lambda x: find_gender_list(x, genders))
ski_list_df

,name,price,detail_link,brand,year,gender
0,Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex
1,Rossignol Black Ops Blazer Skis - Women's 2022,699.95,/skis/rossignol-black-ops-blazer-womens,Rossignol,2022,Women's
2,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's
3,Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex
4,Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex
...,...,...,...,...,...,...
874,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Unisex
875,Atomic Bent Chetler Jr Skis - Little Kids' 202...,199.99,/used/skis/atomic-bent-chetler-jr-little-kids,Atomic,2022,Jr
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Women's
877,Fischer Ranger 99 Ti Skis ​+ Tyrolia Attack2 1...,1048.95,/outlet/used/ski-packages/fischer-ranger-99-ti...,Fischer,0,Unisex


In [76]:
condition= ["used", "Used"]
def find_condition_list(text, condition_list):
    for condition in condition_list:
        if condition in text:
            return "Used" 
        else:
            continue
    return "New"

In [77]:
ski_list_df["condition"]= ski_list_df["name"].apply(lambda x: find_condition_list(x, condition))
ski_list_df

,name,price,detail_link,brand,year,gender,condition
0,Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,New
1,Rossignol Black Ops Blazer Skis - Women's 2022,699.95,/skis/rossignol-black-ops-blazer-womens,Rossignol,2022,Women's,New
2,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,New
3,Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,New
4,Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,New
...,...,...,...,...,...,...,...
874,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Unisex,Used
875,Atomic Bent Chetler Jr Skis - Little Kids' 202...,199.99,/used/skis/atomic-bent-chetler-jr-little-kids,Atomic,2022,Jr,Used
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Women's,Used
877,Fischer Ranger 99 Ti Skis ​+ Tyrolia Attack2 1...,1048.95,/outlet/used/ski-packages/fischer-ranger-99-ti...,Fischer,0,Unisex,Used


In [78]:
ski_list_df["full_link"]= "https://www.evo.com/" + ski_list_df["detail_link"].astype(str)
ski_list_df

,name,price,detail_link,brand,year,gender,condition,full_link
0,Fischer Ranger 102 FR Skis 2022,699.99,/skis/fischer-ranger-102-fr,Fischer,2022,Unisex,New,https://www.evo.com//skis/fischer-ranger-102-fr
1,Rossignol Black Ops Blazer Skis - Women's 2022,699.95,/skis/rossignol-black-ops-blazer-womens,Rossignol,2022,Women's,New,https://www.evo.com//skis/rossignol-black-ops-...
2,Rossignol Black Ops Stargazer Skis - Women's 2022,599.95,/skis/rossignol-black-ops-stargazer-womens,Rossignol,2022,Women's,New,https://www.evo.com//skis/rossignol-black-ops-...
3,Head V-Shape V4 Skis ​+ PR 10 GW Bindings 2022,499.00,/ski-packages/head-v-shape-v4-skis-pr-10-gw-bi...,Head,2022,Unisex,New,https://www.evo.com//ski-packages/head-v-shape...
4,Head V-Shape V6 Skis ​+ PR 11 GW Bindings 2022,599.00,/ski-packages/head-v-shape-v6-skis-pr-11-gw-bi...,Head,2022,Unisex,New,https://www.evo.com//ski-packages/head-v-shape...
...,...,...,...,...,...,...,...,...
874,Atomic Bent Chetler 100 Skis 2022 - Used,599.99,/used/skis/atomic-bent-chetler-100,Atomic,2022,Unisex,Used,https://www.evo.com//used/skis/atomic-bent-che...
875,Atomic Bent Chetler Jr Skis - Little Kids' 202...,199.99,/used/skis/atomic-bent-chetler-jr-little-kids,Atomic,2022,Jr,Used,https://www.evo.com//used/skis/atomic-bent-che...
876,Salomon QST Lux 92 Skis - Women's - Used,499.99,/outlet/used/skis/salomon-qst-lux-92-womens,Salomon,0,Women's,Used,https://www.evo.com//outlet/used/skis/salomon-...
877,Fischer Ranger 99 Ti Skis ​+ Tyrolia Attack2 1...,1048.95,/outlet/used/ski-packages/fischer-ranger-99-ti...,Fischer,0,Unisex,Used,https://www.evo.com//outlet/used/ski-packages/...


In [79]:

#clean_sizes =[]
#clean_dict= {}
#size_dict = {}
for link in ski_list_df["full_link"]:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
    soup= bs(response.content, "html.parser")
    size_dict[link]= []
#     sizes = []
    for div in soup.find_all(class_ = 'pdp-buy-grid-primary js-add-to-cart-loading'):
            ski_size = div.find_all (class_= "pdp-selection-label")
            for i in ski_size:
                size= i.find("span", {"class": "pdp-selection-text pdp-selector"})
                if size != None:
                    temp_string = size.text.strip("\n")
                    if (temp_string.isnumeric()):
                        size_dict[link].append(temp_string)
#     size_dict[link].append(sizes)
    
               
size_dict

{'https://www.evo.com//skis/fischer-ranger-102-fr': ['156',
  '163',
  '170',
  '177',
  '184',
  '191'],
 'https://www.evo.com//skis/rossignol-black-ops-stargazer-womens': ['154',
  '162',
  '170'],
 'https://www.evo.com//ski-packages/head-v-shape-v4-skis-pr-10-gw-bindings': ['156',
  '163',
  '170',
  '177'],
 'https://www.evo.com//ski-packages/head-v-shape-v6-skis-pr-11-gw-bindings': ['156',
  '163',
  '170',
  '177'],
 'https://www.evo.com//ski-packages/head-total-joy-skis-11-gw-slr-bindings-womens': ['148',
  '153',
  '158',
  '163'],
 'https://www.evo.com//skis/rossignol-black-ops-sender': ['164',
  '172',
  '178',
  '186'],
 'https://www.evo.com//skis/k2-mindbender-108ti': ['172', '179', '186', '193'],
 'https://www.evo.com//skis/k2-poacher': ['163', '170', '177', '184'],
 'https://www.evo.com//skis/k2-reckoner-92': ['159', '169', '179'],
 'https://www.evo.com//skis/salomon-qst-lux-92-womens': ['153', '161', '169'],
 'https://www.evo.com//skis/season-forma': ['167', '183'],
 'ht

In [80]:
size_df=pd.DataFrame.from_dict(size_dict, orient="index")
size_df


,0,1,2,3,4,5,6,7,8,9
https://www.evo.com//skis/fischer-ranger-102-fr,156,163,170,177,184,191,None,None,None,None
https://www.evo.com//skis/rossignol-black-ops-stargazer-womens,154,162,170,None,None,None,None,None,None,None
https://www.evo.com//ski-packages/head-v-shape-v4-skis-pr-10-gw-bindings,156,163,170,177,None,None,None,None,None,None
https://www.evo.com//ski-packages/head-v-shape-v6-skis-pr-11-gw-bindings,156,163,170,177,None,None,None,None,None,None
https://www.evo.com//ski-packages/head-total-joy-skis-11-gw-slr-bindings-womens,148,153,158,163,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
https://www.evo.com//used/skis/atomic-bent-chetler-100,172,None,None,None,None,None,None,None,None,None
https://www.evo.com//used/skis/atomic-bent-chetler-jr-little-kids,130,None,None,None,None,None,None,None,None,None
https://www.evo.com//outlet/used/skis/salomon-qst-lux-92-womens,161,None,None,None,None,None,None,None,None,None
https://www.evo.com//outlet/used/ski-packages/fischer-ranger-99-ti-skis-tyrolia-attack2-16-gw-bindings,188,None,None,None,None,None,None,None,None,None


In [82]:
size_df.to_csv("size_data")

In [89]:
size_df.to_excel("size_data.xlsx")

In [ ]:
spec_list= []
dic = {}
for links in ski_list_df["full_link"]:
    response = requests.get(links, headers={'User-Agent': 'Mozilla/5.0'})
    soup= bs(response.content, "html.parser")

    for div in soup.find_all(class_= 'pdp-spec-list'):
            terrain_info = div.find_all(class_='pdp-spec-list-description')
            spec_dict = {}
            specs = ['terrain', 'skis', 'ability_level', 'rocker_type', 'turning_radius', 'core_laminates']
            for idx,s in enumerate(specs):
                try:
                        spec_dict[s] = terrain_info[idx].text.strip()
                except:
                        spec_dict[s] = -9999
                        
            spec_list.append(spec_dict)
            dic[links] = spec_dict

dic

In [ ]:

new_df=pd.DataFrame.from_dict(dic, orient="index")
new_df

In [ ]:
ski_list_df

In [ ]:
detail_page_data = pd.merge(new_df, ski_list_df, left_index=True, right_on = "full_link")
detail_page_data

In [ ]:
detail_page_data.to_csv("detail_page_data")

In [ ]:
import sweetviz

In [ ]:
my_report = sweetviz.analyze([detail_page_data, 'detail_page_data'],'price')
my_report.show_html("detail_page_data.html")